In [ ]:
# INSTALL REQUIRED LIBRARIES
!pip install -q bitsandbytes accelerate transformers datasets peft trl

# LOAD DATASET
import json
from datasets import Dataset

with open("/kaggle/input/PATH OF YOUR DATASET", "r") as f:
    data = json.load(f)

dataset = Dataset.from_list(data).train_test_split(test_size=0.1)

# LOGIN TO HUGGINGFACE
from huggingface_hub import login
login(token="Use your real token")  # Use your real token

# LOAD TOKENIZER
from transformers import AutoTokenizer
model_id = "YOUR MODEL ID"

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# LOAD QUANTIZED MODEL
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

# APPLY LoRA
from peft import get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# FORMAT + TOKENIZE
def format_chat(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}"
    }

formatted = dataset.map(format_chat)

def tokenize(example):
    encoding = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    encoding["labels"] = encoding["input_ids"].copy()
    return encoding

tokenized = formatted.map(tokenize)

# ✅ DATALOADER
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
# TRAINING ARGS (Optimized for Kaggle Free Tier)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,  # Slows down memory usage
    learning_rate=2e-4,
    num_train_epochs=1,
    max_steps=500,                 # Stops after 500 steps (~30–40 mins)
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none",
    fp16=True,
    logging_first_step=True,
    disable_tqdm=False
)

# TRAINER
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    data_collator=data_collator
)

# START TRAINING
trainer.train()

# SAVE LoRA ADAPTER ONLY (lightweight)
model.save_pretrained("YOUR DESIRED NAME")
tokenizer.save_pretrained("YOUR DESIRED NAME")